In [ ]:
from __future__ import print_function
%matplotlib inline
import scipy as sp
from scipy.interpolate import interp1d
import iminuit
from matplotlib.pyplot import errorbar, plot, contour, hist2d, grid, hist
from scipy import random
import emcee
import corner

In [ ]:
data=sp.loadtxt('data/SCPUnion2.1_mu_vs_z.txt',usecols=(1,2,3))
z = data[:,0]
M = data[:,1]
dM = data[:,2]
## plot data:
errorbar(z, M, yerr = dM, fmt = "o")

In [ ]:
def hubble(z, H0, Om, OL):
    H = H0*sp.sqrt(Om*(1+z)**3 + OL + (1-Om-OL)*(1+z)**2)
    return H

In [ ]:
nbins=10000
zmax = 2.
zs = sp.arange(nbins, dtype=float)/(nbins-1)*zmax
dz = zmax/nbins
c = 300000. ## km/s
def D_L(H0, Om, OL):
    chi = (dz*H0/hubble(zs, H0, Om, OL)).cumsum()
    chi = interp1d(zs, chi)(z)
    Ok = 1-Om-OL
    if Ok==0:
        return c/H0*(1+z)*chi
    if Ok > 0:
        return c/H0*(1+z)*sp.sinh(sp.sqrt(Ok)*chi)/sp.sqrt(Ok)
    if Ok<0:
        return c/H0*(1+z)*sp.sin(sp.sqrt(-Ok)*chi)/sp.sqrt(-Ok)
    

In [ ]:
## distance modulus
def mu(H0, Om, OL):
    d_pc = D_L(H0, Om, OL)*1e6
    return 5*sp.log10(d_pc)-5

In [ ]:
errorbar(z, M, yerr = dM, fmt = "o")
plot(z, mu(70., 0.3, 0.7),"o")
grid()

In [ ]:
def chi2(H0, Om, OL):
    res = (M - mu(H0, Om, OL))/dM
    return (res**2).sum()

In [ ]:
mig = iminuit.Minuit(chi2, H0=70., Om=0.3, 
        OL=0.7, error_H0=1, error_Om=0.1, error_OL=0.1,
        errordef=1)

In [ ]:
fmin = mig.migrad()
print("Number of data points: ",len(M))

In [ ]:
nOm = 40
Om = 0.5*sp.arange(nOm, dtype=float)/(nOm-1)
nOL = 40
OL = 0.2+1.*sp.arange(nOL, dtype=float)/(nOL-1)
chi2_scan = sp.zeros((nOL, nOm))
for i in range(nOm):
    print('\rdoing chi2 scan {}%'.format(round(i*100./nOm,2)), end='')
    for j in range(nOL):
        H2 = Om[i]*(1+zs)**3 + OL[j] + (1-Om[i]-OL[j])*(1+zs)**2
        if (H2<=0).any():
            chi2_scan[j, i] = sp.inf
            continue
        mig_scan = iminuit.Minuit(chi2, H0=70., Om=Om[i], 
        OL=OL[j], error_H0=1, error_Om=0.1, error_OL=0.1,
        errordef=1, fix_Om=True, fix_OL=True, print_level=0, limit_H0 = (0, None))
        mig_scan.migrad()
        chi2_scan[j, i] = mig_scan.fval-mig.fval
print('\ndone')

In [ ]:
#contourf(Om, OL, sp.log10(chi2_scan))
#colorbar()
contour(Om, OL, chi2_scan, levels=[2.3, 6., 11.8], colors="k")
plot(Om,1-Om,"k--")

In [ ]:
NMC = 500
Om_mc = sp.zeros(NMC)
OL_mc = sp.zeros(NMC)
mig_mc = iminuit.Minuit(chi2, H0=70., Om=0.3, 
        OL=0.7, error_H0=1, error_Om=0.1, error_OL=0.1,
        errordef=1, print_level=0)
for i in range(NMC):
    ## get the distance modulus for the best fit parameters
    M = mu(mig.values['H0'], mig.values['Om'], mig.values['OL'])
    M += dM*random.randn(len(M))
    fmin = mig_mc.migrad()
    print("\riteration: {} of {}, Om: {}, OL: {}".format(i, NMC, mig_mc.values['Om'], mig_mc.values['OL']), end='')
    Om_mc[i] = mig_mc.values['Om']
    OL_mc[i] = mig_mc.values['OL']
    

In [ ]:
contour(Om, OL, chi2_scan, levels=[2.3, 6, 11.8], colors="k")
plot(Om,1-Om,"k--")
plot(Om_mc, OL_mc, "o")

In [ ]:
data=sp.loadtxt('data/SCPUnion2.1_mu_vs_z.txt',usecols=(1,2,3))
z = data[:,0]
M = data[:,1]
dM = data[:,2]

In [ ]:
def lnprob(pars):
    H0 = pars[0]
    Om = pars[1]
    OL = pars[2]
    if H0<0:
        return -sp.inf
    H2 = H0**2*(Om*(1+z)**2 + OL + (1-Om-OL)*(1+z)**2)
    if (H2<0).any():
        return -sp.inf
    
    return -0.5*chi2(H0, Om, OL)

In [ ]:
ndim, nwalkers = 3, 50
Om0 = 0.3 + 0.01*random.randn(nwalkers)
OL0 = 0.7 + 0.01*random.randn(nwalkers)
H0 = 70. + 0.01*random.rand(nwalkers)
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

In [ ]:
pos, prob, state = sampler.run_mcmc(sp.array([H0, Om0, OL0]).T, 500)

In [ ]:
for i in range(nwalkers):
    plot(sampler.chain[i,:,2],"k")

In [ ]:
_=hist2d(sampler.chain[:,100:,1].flatten(), sampler.chain[:,100:,2].flatten(), bins=40)

In [ ]:
_=hist(sampler.chain[:,100:,1].flatten(), bins=100)

In [ ]:
samples = sampler.chain[:,100:,:].reshape([-1, ndim])

In [ ]:
corner.corner(samples[:,:])

In [ ]:
contour(Om, OL, chi2_scan, levels=[2.3, 6, 11.8], colors="k")
plot(Om,1-Om,"k--")
plot(samples[:,1], samples[:,2], "o", alpha=0.05)